In [2]:
import sys
!{sys.executable} -m pip install pyspark
import pyspark
print(pyspark.__version__)
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("Customer Session Hours").getOrCreate()

# Sample data (as given in the problem)
data = [
    ('c001', 1, '07:00:00'),
    ('c001', 0, '09:30:00'),
    ('c001', 1, '12:00:00'),
    ('c001', 0, '14:30:00'),
    ('c002', 1, '08:00:00'),
    ('c002', 0, '09:30:00'),
    ('c002', 1, '11:00:00'),
    ('c002', 0, '12:30:00'),
    ('c002', 1, '15:00:00'),
    ('c002', 0, '16:30:00'),
    ('c003', 1, '09:00:00'),
    ('c003', 0, '10:30:00'),
    ('c004', 1, '10:00:00'),
    ('c004', 0, '10:30:00'),
    ('c004', 1, '14:00:00'),
    ('c004', 0, '15:30:00'),
    ('c005', 1, '10:00:00'),
    ('c005', 0, '14:30:00'),
    ('c005', 1, '15:30:00'),
    ('c005', 0, '18:30:00')
]

# Create a DataFrame
columns = ["cust_id", "state", "timestamp"]
df = spark.createDataFrame(data, columns)

# Convert 'timestamp' to a proper timestamp type (using 'HH:MM:SS' format)
df = df.withColumn("timestamp", F.col("timestamp").cast("timestamp"))

# Define a window specification to partition by cust_id and order by timestamp
window_spec = Window.partitionBy("cust_id").orderBy("timestamp")

# 1. Calculate the previous timestamp for each row using LAG() function
df_with_lag = df.withColumn("prev_timestamp", F.lag("timestamp").over(window_spec))

# 2. Filter out rows where state is 0 (session ends)
session_ends = df_with_lag.filter(df_with_lag.state == 0)

# 3. Calculate session duration in minutes (difference between the session end time and start time)
session_duration = session_ends.withColumn(
    "session_duration_minutes",
    (F.unix_timestamp("timestamp") - F.unix_timestamp("prev_timestamp")) / 60
)

# 4. Group by cust_id and calculate the total active time in hours
result = session_duration.groupBy("cust_id").agg(
    F.sum("session_duration_minutes").alias("total_active_minutes")
)

# Convert total active minutes to hours
result = result.withColumn("total_active_hours", F.col("total_active_minutes") / 60)

# Show the final result
result.show()





[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
3.5.4


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/19 22:38:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+--------------------+------------------+
|cust_id|total_active_minutes|total_active_hours|
+-------+--------------------+------------------+
|   c001|               300.0|               5.0|
|   c002|               270.0|               4.5|
|   c003|                90.0|               1.5|
|   c004|               120.0|               2.0|
|   c005|               450.0|               7.5|
+-------+--------------------+------------------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53270)
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/Bubu/DE/de-azure-spotify-project/sptfy-env/lib/python3.11/site

In [ ]:
import sys
print(sys.executable)


/Users/Bubu/DE/de-azure-spotify-project/sptfy-env/bin/python


25/02/20 03:10:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 992757 ms exceeds timeout 120000 ms
25/02/20 03:10:15 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/20 03:10:19 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$